In [2]:
fusers=['julie', 'eva', 'rebecca', 'cinthia', 'caroline','ariel', 'lynn', 'judith']
musers=['josh', 'jake', 'matthew', 'luke', 'aaron', 'ted']
chat1=[]
chat2=[]

In [3]:
#parsing data
f=open("data/Mar07_GroupB.txt","r")
for line in f:
    x=line.split("):")
    u=x[0].split()[0]
    if u=="moderator":
      continue
    sentence=x[1].strip()
    chat1.append((u,sentence))

In [ ]:
f=open("data/Mar06_GroupA.txt","r")
for line in f:
    x=line.split("):")
    u=x[0].split()[0]
    if u=="moderator":
      continue
    sentence=x[1].strip()
    chat2.append((u,sentence))

In [5]:
f=open("data/English_ANEW_long_DEFT.txt","r")
anew={}
x=5
for line in f:
    if(x!=0):
        x-=1
        continue
    temp=line.split()
    anew[temp[2]]=float(temp[1])
    

In [6]:
#core nlp lib spaCy
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
#word lists from https://elib.uni-stuttgart.de/bitstream/11682/9665/1/ausarbeitung.pdf
diminishers=["almost","barely","bit","exiguous","faintly","fairly","few","fewer","hardly","insignificantly","kinda","less","little","maybe","marginally","moderately","modicum","mostly","nearly","negligibly","partly","partially","practically","quite","rather","rarely","relatively","reasonably","scanty","scarcely","slightly","some","somewhat","sparsely","tolerably","triflingly","virtually"]
intensifiers=["absolutely","altogether","amazingly","astoundingly","awfully","badly","decidedly","bitterly","bloody","colossally","completely","damn","deeply","drastically","dreadfully","entirely","enormously","especially","exceptionally","excessively","extraordinarily","extremely","fantastically","freaking","frightfully","fucking","fully","greatly","hella","highly","incredibly","insanely","intensely","immensely","largely","literally","lot","lots","massive","mightily","more","outrageously","particularly","perfectly","phenomenally","pretty","radically","real","really","remarkably","purely","so","soo","sooo","super","supremely","surpassingly","strikingly","strongly","terribly","terrifically","totally","thoroughly","truly","unusually","utterly","very","wicked"]

In [8]:
#class for easier access to counts + avg polarity
class entry:
    def __init__(self,c,val):
        self.count=c
        self.lst=[val]
    def __str__(self):
        return str(self.count)+':'+str(self.lst)
    def __repr__(self):
        return str(self.count)+':'+str(self.lst)
    def __lt__(self, other):
        return self.count<other.count

In [9]:
#main calc for chats, chat1 for the users-message dictionary, p is the list of proper nouns (only for chat)
def calc(chat1,p):
    fchat={}
    mchat={}
    propers=p
    lastProper=""
    lastNoun=""
    for message in chat1:
        doc = nlp(message[1])
        nouns=[]
        pronouns=[]
        proNounReplacers=[]
        for token in doc:
            #find noun subjects
            if "nsubj" == token.dep_:
                #pronoun replacement
                if "PRON"== token.pos_:
                    if token.text.lower()=="she" or token.text.lower()=="he":
                        if(token.text.lower() in pronouns):
                            proNounReplacers.append(proNounReplacers[pronouns.index(token.text.lower())])
                        else:
                            proNounReplacers.append(lastProper)
                            pronouns.append(token.text.lower())
                    elif token.text.lower()=="it":
                        proNounReplacers.append(lastNoun)
                    if(token.text.lower()!="i"):
                        nouns.append(token)
                #keeping track of new nouns and proper nouns to use in pronoun replacement
                elif "PROPN"== token.pos_ and token.text.lower() in propers:
                    lastProper=token.text.lower()
                    nouns.append(token)
                else:
                    lastNoun=token.text.lower()
                    nouns.append(token)
            elif "PROPN"== token.pos_ and token.text.lower() in propers:
                    lastProper=token.text.lower()
            elif "NOUN"== token.pos_:
                lastNoun=token.text
                
        #if len(proNounReplacers)!=0:
        #    print(proNounReplacers)
        
        #iterate through the identified subjects
        for noun in nouns:
                if noun.head.pos_=="NOUN":
                    pass
                #look at noun and its head, english in S-V-O structure, looking at stuff of V
                queue=[noun.head,noun]
                #interesting an relevant words
                coolwords=[]
                #tree traversal
                while(len(queue)!=0):
                    node=queue.pop(0)
                    # only add adjectives, adverbs, negation, and attributes
                    if node.pos_=="ADV" or node.pos_=="ADJ" or node.dep_=="attr" or node.dep_=="neg":
                        coolwords.append(node)
                    for child in node.children:
                        if child!=noun and child.dep_!="ROOT":
                            queue.append(child)
                #print(noun.text+" "+str(coolwords))
                
                numIntensifiers=0
                numDiminishers=0
                polarity=0.0
                count=0
                #compute polarity of words associated with the subject
                for word in coolwords:
                    temp=word.text.lower()
                    if temp in intensifiers:
                        numIntensifiers+=1
                    elif temp in diminishers:
                        numDiminishers+=1
                    else:
                        #look in anew, if not foun tried the lemmatized version
                        if temp in anew.keys():
                            polarity+=anew[temp]
                            count+=1
                        elif word.lemma_.lower() in anew.keys():
                            polarity+=anew[word.lemma_.lower()]
                            count+=1
                #print(numIntensifiers)
                #print(numDiminishers)
                if(count!=0):
                    #computer average polarity
                    polarity=polarity/count
                    if(polarity!=5.0):
                        #add intensifier and diminsher influences
                        direction=(polarity-5)/abs(polarity-5)
                        polarity+=(.2*numIntensifiers*direction)
                        polarity+=(-.2*numDiminishers*direction)
                    else:
                        polarity=5.0
                        
                if(polarity!=0.0):
                    polarity-=5
                    polarity=abs(polarity)
                #replace the pronoun before storing
                if(len(pronouns)>=1 and noun.text.lower()==pronouns[0]):
                    pronouns.pop(0)
                    temp=proNounReplacers.pop(0)
                else:
                    temp=noun.text.lower()
                #store the data
                if(temp!=None):
                    if(message[0] in fusers):
                        if(message[0] in fchat.keys()):
                            if(temp in fchat[message[0]].keys()):
                                fchat[message[0]][temp].count+=1
                                fchat[message[0]][temp].lst.append(polarity)
                            else:
                                fchat[message[0]][temp]=entry(1,polarity)
                        else:
                            fchat[message[0]]={}
                            fchat[message[0]][temp]=entry(1,polarity)
                    else:
                        if(message[0] in mchat.keys()):
                            if(temp in mchat[message[0]].keys()):
                                mchat[message[0]][temp].count+=1
                                mchat[message[0]][temp].lst.append(polarity)
                            else:
                                mchat[message[0]][temp]=entry(1,polarity)
                        else:
                            mchat[message[0]]={}
                            mchat[message[0]][temp]=entry(1,polarity)   
    return (fchat,mchat)

In [18]:
x=calc(chat1,["jade","penny","ryan","evan"])
fchat=x[0]
mchat=x[1]
#compute average polarity on topics per each user
fresults={}
for user in fchat.keys():
    sum=0
    for topic in fchat[user].keys():
        subsum=0
        for x in fchat[user][topic].lst:
            subsum+=x
        if((fchat[user][topic].count-fchat[user][topic].lst.count(0.0))!=0):
            #remove topics mentions that have no sentiment
            avg=subsum/(fchat[user][topic].count-fchat[user][topic].lst.count(0.0))
            sum+=avg+(fchat[user][topic].count/100)*avg
    fresults[user]=sum/len(fchat[user].keys())
print("f1")
for val in fresults.values():
    print(val)

#repeated as above
mresults={}
for user in mchat.keys():
    sum=0
    for topic in mchat[user].keys():
        subsum=0
        for x in mchat[user][topic].lst:
            subsum+=x
        if((mchat[user][topic].count-mchat[user][topic].lst.count(0.0))!=0):
            avg=subsum/(mchat[user][topic].count-mchat[user][topic].lst.count(0.0))
            sum+=avg+(mchat[user][topic].count/100)*avg
    mresults[user]=sum/len(mchat[user].keys())
print("m1")
for val in mresults.values():
    print(val)

x=calc(chat2,["alan","dwight","clark","brent"])
fchat=x[0]
mchat=x[1]
fresults={}
for user in fchat.keys():
    sum=0
    for topic in fchat[user].keys():
        subsum=0
        for x in fchat[user][topic].lst:
            subsum+=x
        if((fchat[user][topic].count-fchat[user][topic].lst.count(0.0))!=0):
            avg=subsum/(fchat[user][topic].count-fchat[user][topic].lst.count(0.0))
            sum+=avg+(fchat[user][topic].count/100)*avg
    fresults[user]=sum/len(fchat[user].keys())
print("f2")
for val in fresults.values():
    print(val)

mresults={}
for user in mchat.keys():
    sum=0
    for topic in mchat[user].keys():
        subsum=0
        for x in mchat[user][topic].lst:
            subsum+=x
        if((mchat[user][topic].count-mchat[user][topic].lst.count(0.0))!=0):
            avg=subsum/(mchat[user][topic].count-mchat[user][topic].lst.count(0.0))
            sum+=avg+(mchat[user][topic].count/100)*avg
    mresults[user]=sum/len(mchat[user].keys())
print("m2")
for val in mresults.values():
    print(val)
'''
results:
f1
1.231325072463768
0.8457546398046398
0.7267456790123455
m1
0.8579449333333332
0.9151500000000001
0.43073947368421056
f1
0.8091413333333334
0.5163308823529412
1.0245846666666667
1.0822920454545453
0.7764652173913043
m2
0.8668381944444445
0.9134348214285714
'''

f1
1.231325072463768
0.8457546398046398
0.7267456790123455
m1
0.8579449333333332
0.9151500000000001
0.43073947368421056
f1
0.8091413333333334
0.5163308823529412
1.0245846666666667
1.0822920454545453
0.7764652173913043
f2
0.8668381944444445
0.9134348214285714
0.6631181818181818


"\nresults:\n{'lynn': 1.2293985507246374, 'ariel': 0.7148748473748474, 'judith': 0.7030864197530865}\n{'aaron': 0.8020800000000001, 'luke': 0.8881250000000002, 'ted': 0.34986842105263155}\n\n{'julie': 0.8397333333333333, 'caroline': 0.460735294117647, 'rebecca': 1.0081333333333335, 'eva': 1.0427272727272725, 'cinthia': 0.5882608695652174}\n{'josh': 0.8320138888888889, 'jake': 0.9204166666666669, 'matthew': 0.48727272727272736}\n"

In [19]:
#using twitter data
import pandas as pd
import io
data = pd.read_csv('data/gender-classifier-DFE-791531.csv',encoding = "ISO-8859-1")
data.head(1)
data=data.drop(columns=['tweet_count', 'gender:confidence', 'description', 'fav_number', 'link_color', 'name', 'retweet_count', 'sidebar_color', "_unit_id","_unit_state","_golden","_trusted_judgments","_last_judgment_at","profile_yn","profile_yn:confidence","created","profile_yn_gold","profileimage","gender_gold","tweet_id","tweet_location","user_timezone","tweet_coord","tweet_created"],axis=1)
#delete non gendered rows
for index,row in data.iterrows():
  if row['gender']!='male' and row['gender']!='female':
    data.drop(index,inplace=True)
dataDict=data.to_dict()

In [20]:
fchat={}
mchat={}
lastProper=""
lastNoun=""
#same as before, but with no pronoun name lists
for index in dataDict['gender'].keys():
    doc = nlp(dataDict['text'][index])
    nouns=[]
    pronouns=[]
    proNounReplacers=[]
    for token in doc:
        if "nsubj" == token.dep_:
            if "PRON"== token.pos_:
                if token.text.lower()=="she" or token.text.lower()=="he":
                    if(token.text.lower() in pronouns):
                        proNounReplacers.append(proNounReplacers[pronouns.index(token.text.lower())])
                    else:
                        proNounReplacers.append(lastProper)
                        pronouns.append(token.text.lower())
                elif token.text.lower()=="it":
                    proNounReplacers.append(lastNoun)
                if(token.text.lower()!="i"):
                    nouns.append(token)
            elif "PROPN"== token.pos_ :
                lastProper=token.text.lower()
                nouns.append(token)
            else:
                lastNoun=token.text.lower()
                nouns.append(token)
        elif "PROPN"== token.pos_:
                lastProper=token.text.lower()
        elif "NOUN"== token.pos_:
            lastNoun=token.text

    for noun in nouns:
            if noun.head.pos_=="NOUN":
                pass
            queue=[noun.head,noun]
            coolwords=[]
            while(len(queue)!=0):
                node=queue.pop(0)
                if node.pos_=="ADV" or node.pos_=="ADJ" or node.dep_=="attr" or node.dep_=="neg":
                    coolwords.append(node)
                for child in node.children:
                    if child!=noun and child.dep_!="ROOT":
                        queue.append(child)
            #print(noun.text+" "+str(coolwords))
            numIntensifiers=0
            numDiminishers=0
            polarity=0.0
            count=0
            for word in coolwords:
                temp=word.text.lower()
                if temp in intensifiers:
                    numIntensifiers+=1
                elif temp in diminishers:
                    numDiminishers+=1
                else:
                    if temp in anew.keys():
                        polarity+=anew[temp]
                        count+=1
                    elif word.lemma_.lower() in anew.keys():
                        polarity+=anew[word.lemma_.lower()]
                        count+=1
            #print(numIntensifiers)
            #print(numDiminishers)
            if(count!=0):
                polarity=polarity/count
                if(polarity!=5.0):
                    direction=(polarity-5)/abs(polarity-5)
                    #print(direction)
                    polarity+=(.2*numIntensifiers*direction)
                    polarity+=(-.2*numDiminishers*direction)
                else:
                    polarity=5.0
            if(polarity!=0.0):
                polarity-=5
                polarity=abs(polarity)
            if(len(pronouns)>=1 and noun.text.lower()==pronouns[0]):
                pronouns.pop(0)
                temp=proNounReplacers.pop(0)
            else:
                temp=noun.text.lower()
            if(temp!=None):
                if(dataDict['gender'][index]=="female"):
                    if(index in fchat.keys()):
                        if(temp in fchat[index].keys()):
                            fchat[index][temp].count+=1
                            fchat[index][temp].lst.append(polarity)
                        else:
                            fchat[index][temp]=entry(1,polarity)
                    else:
                        fchat[index]={}
                        fchat[index][temp]=entry(1,polarity)
                else:
                    if(index in mchat.keys()):
                        if(temp in mchat[index].keys()):
                            mchat[index][temp].count+=1
                            mchat[index][temp].lst.append(polarity)
                        else:
                            mchat[index][temp]=entry(1,polarity)
                    else:
                        mchat[index]={}
                        mchat[index][temp]=entry(1,polarity)   

In [22]:
#same as before
fresults={}
for user in fchat.keys():
    sum=0
    for topic in fchat[user].keys():
        subsum=0
        for x in fchat[user][topic].lst:
            subsum+=x
        if((fchat[user][topic].count-fchat[user][topic].lst.count(0.0))!=0):
            avg=subsum/(fchat[user][topic].count-fchat[user][topic].lst.count(0.0))
            sum+=avg+(fchat[user][topic].count/100)*avg
    fresults[user]=sum/len(fchat[user].keys())

mresults={}
for user in mchat.keys():
    sum=0
    for topic in mchat[user].keys():
        subsum=0
        for x in mchat[user][topic].lst:
            subsum+=x
        if((mchat[user][topic].count-mchat[user][topic].lst.count(0.0))!=0):
            avg=subsum/(mchat[user][topic].count-mchat[user][topic].lst.count(0.0))
            sum+=avg+(mchat[user][topic].count/100)*avg
    mresults[user]=sum/len(mchat[user].keys())

#remove users with no detected polarity in tweet
favg=0
count1=0
for user in fresults.keys():
    favg+=fresults[user]
    if fresults[user]==0:
        count1+=1
favg/=len(fresults.keys())-count1

mavg=0
count2=0
for user in mresults.keys():
    mavg+=mresults[user]
    if mresults[user]==0:
        count2+=1
mavg/=len(mresults.keys())-count2

print(favg)

print(mavg)

#F with 0's: 0.5698763670166223
#M with 0's: 0.5486616276140085
#F without 0's: 1.2472353997127799
#M without 0's: 1.1472725356935882

1.2640142275494708
1.1645548289199301


In [8]:
#showcase
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy
doc = nlp("max is cool and mack is bad")
displacy.render(doc, style='dep')
nouns=[]
pronouns=[]
proNounReplacers=[]
lastProper=""
lastNoun=""
for token in doc:
    if "nsubj" == token.dep_:
        if "PRON"== token.pos_:
            if token.text.lower()=="she" or token.text.lower()=="he":
                if(token.text.lower() in pronouns):
                    proNounReplacers.append(proNounReplacers[pronouns.index(token.text.lower())])
                else:
                    proNounReplacers.append(lastProper)
                    pronouns.append(token.text.lower())
            elif token.text.lower()=="it":
                proNounReplacers.append(lastNoun)
            if(token.text.lower()!="i"):
                nouns.append(token)
        elif "PROPN"== token.pos_ :
            lastProper=token.text.lower()
            nouns.append(token)
        else:
            lastNoun=token.text.lower()
            nouns.append(token)
    elif "PROPN"== token.pos_:
            lastProper=token.text.lower()
    elif "NOUN"== token.pos_:
        lastNoun=token.text

for noun in nouns:
        if noun.head.pos_=="NOUN":
            pass
        queue=[noun.head,noun]
        coolwords=[]
        while(len(queue)!=0):
            node=queue.pop(0)
            if node.pos_=="ADV" or node.pos_=="ADJ" or node.dep_=="attr" or node.dep_=="neg":
                coolwords.append(node)
            for child in node.children:
                if child!=noun and child.dep_!="ROOT":
                    queue.append(child)
        print(noun.text+" "+str(coolwords))

max [cool, bad]
mack [bad]
